In [1]:
import stl
from stl import mesh

import numpy as np
import scipy.signal as signal 
from scipy.signal import fftconvolve
from scipy.spatial import distance_matrix
import soundfile as sf
import pyroomacoustics as pra

In [2]:
%matplotlib notebook

In [3]:
path_to_musis_stl_file = "data/smaller_slicedup_OC_evenmore.stl"

#path_to_musis_stl_file = './INRIA_MUSIS.stl'


material = pra.Material(energy_absorption=0.4, scattering=0.5)

# with numpy-stl
the_mesh = mesh.Mesh.from_file(path_to_musis_stl_file)
ntriang, nvec, npts = the_mesh.vectors.shape
size_reduc_factor = 1  # to get a realistic room size (not 3km)

In [4]:
# create one wall per triangle
walls = []
for w in range(ntriang):
    walls.append(
        pra.wall_factory(
            the_mesh.vectors[w].T / size_reduc_factor,
            material.energy_absorption["coeffs"],
            material.scattering["coeffs"],
        )
    )

In [ ]:
source_sound = np.random.normal(0,1,1000)
source_sound *= signal.hanning(source_sound.size)

source_position = np.array([-0.2, 0, -1])
central_mic_pos = np.array([-0.75, 01.66, -0.5])

source_position = source_position.tolist()
central_mic_pos = central_mic_pos.tolist()

In [ ]:
call_durn = 0.004 # seconds
fs = 192000
t = np.linspace(0,call_durn,int(fs*call_durn))
bat_call = signal.chirp(t,90000, t[-1],30000)
bat_call *= signal.windows.tukey(bat_call.size, 0.95)

In [ ]:
room = pra.Room(walls, fs=fs, max_order=1, ray_tracing=True, air_absorption=True).add_source(
                                    [0.05, 0.50, 0.05], signal=bat_call).add_microphone_array(
                                    np.c_[[-1.1, 0.9, 0.1], [-1.1, 0.9,0.3]])

In [ ]:
room.rt_args

In [ ]:
room.image_source_model()
room.ray_tracing()
room.compute_rir()
room.plot_rir()


In [ ]:
mic0_signal = fftconvolve(room.rir[0][0], bat_call)
mic1_signal = fftconvolve(room.rir[1][0], bat_call)
plt.figure()
a0 = plt.subplot(211)
plt.specgram(mic0_signal, Fs=fs, NFFT=192, noverlap=96);
plt.subplot(212, sharex=a0)
plt.plot(np.linspace(0,mic0_signal.size/fs,mic0_signal.size), mic0_signal)


In [ ]:
stereo = np.zeros((np.max([mic0_signal.size, mic1_signal.size]),2))
stereo[:mic0_signal.size,0] = mic0_signal
stereo[:mic1_signal.size,1] = mic1_signal

In [ ]:
sf.write('mic01_signal.wav', stereo, fs)


Let's try to extract the source-level using the distance. How badly will this fail when there is reverberation? 


In [ ]:
acc0 = signal.correlate(stereo[:,0], stereo[:,0])
acc1 = signal.correlate(stereo[:,1], stereo[:,1])
plt.figure()
plt.subplot(211)
plt.plot(acc0)
plt.subplot(212)
plt.plot(acc1)